In [1]:
import torch
from models import (
    NoSkipConnectionSupervenientFeatureNetwork,
    SkipConnectionSupervenientFeatureNetwork,
    DownwardSmileMIEstimator
)
from datasets import BitStringDataset, ECoGDataset
import lovely_tensors as lt
import wandb
import tqdm
from trainers import train_feature_network

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# training model A's on ecog

In [2]:
import torch
from models import SkipConnectionSupervenientFeatureNetwork
from datasets import ECoGDataset
from trainers import train_feature_network

dataset = ECoGDataset()

device = 'cuda'

trainloader = torch.utils.data.DataLoader(dataset, batch_size=1000, shuffle=True)

for seed in range(4):

    torch.manual_seed(seed)

    config = {
        "torch_seed": seed,
        "dataset_type": "ecog",
        "num_atoms": 64,
        "batch_size": 1000,
        "train_mode": True,
        "train_model_B": False,
        "adjust_Psi": False,
        "clip": 5,
        "feature_size": 3,
        "epochs": 30,
        "start_updating_f_after": 300,
        "update_f_every_N_steps": 5,
        "minimize_neg_terms_until": 9999999999999,
        "downward_critics_config": {
            "hidden_sizes_v_critic": [512, 1024, 1024, 512],
            "hidden_sizes_xi_critic": [512, 512, 512],
            "critic_output_size": 32,
            "lr": 1e-2,
            "bias": True,
            "weight_decay": 0,
        },
        
        "decoupled_critic_config": {
            "hidden_sizes_encoder_1": [512, 512, 512],
            "hidden_sizes_encoder_2": [512, 512, 512],
            "critic_output_size": 32,
            "lr": 1e-2,
            "bias": True,
            "weight_decay": 0,
        },
        "feature_network_config": {
            "hidden_sizes": [256, 256, 256, 256, 256],
            "lr": 1e-3,
            "bias": True,
            "weight_decay": 1e-3,
        }
    }


    skip_model = SkipConnectionSupervenientFeatureNetwork(
        num_atoms=config['num_atoms'],
        feature_size=config['feature_size'],
        hidden_sizes=config['feature_network_config']['hidden_sizes'],
        include_bias=config['feature_network_config']['bias'],
    ).to(device)

    # project_name = "NEURIPS-FINAL-finding-emergent-ecog-features"
    # model_dir_prefix='NEURIPS-FINAL-ecog-f_network-A'
    project_name = 'meme'
    model_dir_prefix = 'meme_modle'
    skip_model = train_feature_network(
        config=config,
        trainloader=trainloader,
        feature_network_training=skip_model,
        project_name=project_name,
        model_dir_prefix='NEURIPS-FINAL-ecog-f_network-A'
    )


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmcsharry. Use `wandb login --relogin` to force relogin


Training:  13%|█▎        | 4/30 [05:57<38:42, 89.31s/it]


KeyboardInterrupt: 

# Oher shit

In [ ]:
import torch
from models import SkipConnectionSupervenientFeatureNetwork
from datasets import ECoGDataset
from trainers import train_feature_network

seed = 0

torch.manual_seed(seed)


dataset = ECoGDataset()

device = 'cuda'

trainloader = torch.utils.data.DataLoader(dataset, batch_size=1000, shuffle=True)



config = {
    "torch_seed": seed,
    "dataset_type": "ecog",
    "num_atoms": 64,
    "batch_size": 1000,
    "train_mode": True,
    "train_model_B": False,
    "adjust_Psi": True,
    "clip": 5,
    "feature_size": 3,
    "epochs": 20,
    "start_updating_f_after": 500,
    "update_f_every_N_steps": 5,
    "minimize_neg_terms_until": 0,
    "downward_critics_config": {
        "hidden_sizes_v_critic": [512, 1024, 1024, 512],
        "hidden_sizes_xi_critic": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "decoupled_critic_config": {
        "hidden_sizes_encoder_1": [512, 1024, 1024, 512],
        "hidden_sizes_encoder_2": [512, 1024, 1024, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "feature_network_config": {
        "hidden_sizes": [256, 256, 256, 256, 256],
        "lr": 1e-4,
        "bias": True,
        "weight_decay": 1e-3,
    }
}


model_path = "/vol/bitbucket/dm2223/info-theory-experiments/models/feature_network_earthy-sun-6.pth"


feature_network = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_network_config']['hidden_sizes'],
    include_bias=config['feature_network_config']['bias'],
).to(device)

feature_network.load_state_dict(torch.load(model_path))

project_name = "training an ecog network on full Psi after infomin"


feature_network = train_feature_network(
    config=config,
    trainloader=trainloader,
    feature_network_training=feature_network,
    project_name=project_name
)
